In [ ]:
import os
import numpy as np
import mne
from mne.io import read_raw_cnt
from mne.preprocessing import ICA, create_eog_epochs
from mne.channels import make_standard_montage
from mne import Epochs, concatenate_epochs
import matplotlib.pyplot as plt

In [ ]:
# Set directories
field_trip_directory = r'C:\Users\Mahmud\Documents\fieldtrip-20220729'
my_directory = r'E:\Research\EEG_Data\ano'
my_save_directory = input("Enter the directory where you want to save the preprocessed data: ")

# Set participant ID
participant_id = input("Input participant ID number: ")

# Read file list
file_pattern = f'{participant_id}*.cnt'
cnt_files = [f for f in os.listdir(my_directory) if f.endswith('.cnt') and f.startswith(participant_id)]

# Define trial markers and parameters
trial_markers = np.arange(0, 121)
my_prestim = 3  # seconds
my_poststim = 4  # seconds
block_size = 7  # block size in seconds for visualization

# Load the electrode layout
elec_aligned = mne.channels.make_standard_montage('standard_1005')

# Initialize a list to store processed data
processed_data_list = []

# Loop through each file and preprocess
for cnt_file in cnt_files:
    file_path = os.path.join(my_directory, cnt_file)
    print(f"Processing file: {file_path}")
    
    # Load the raw CNT file
    raw = read_raw_cnt(file_path, preload=True)
    
    # Drop unwanted channels
    drop_channels = ['M1', 'M2', 'EMG', 'REF', 'COMNT', 'SCALE']
    raw.drop_channels([ch for ch in drop_channels if ch in raw.ch_names])
    
    # Apply high-pass filter
    raw.filter(l_freq=0.1, h_freq=None, method='iir', iir_params=dict(order=1, ftype='butter'))
    
    # Apply low-pass filter
    raw.filter(l_freq=None, h_freq=100, method='iir', iir_params=dict(order=1, ftype='butter'))
    
    # Define events
    events = mne.find_events(raw, stim_channel='STI 014', output='onset', min_duration=0.002)
    
    # Create epochs based on events
    event_id = {str(i): i for i in trial_markers}
    epochs = mne.Epochs(raw, events, event_id=event_id, tmin=-my_prestim, tmax=my_poststim, baseline=None, preload=True)
    
    # Resample epochs to 500 Hz
    epochs_resampled = epochs.copy().resample(500)
    
    processed_data_list.append(epochs_resampled)

In [ ]:
# Concatenate all epochs from different files
all_epochs = concatenate_epochs(processed_data_list)

# Visualize the data to identify bad channels
all_epochs.plot(n_channels=64, scalings='auto', block=True)

# List bad channels manually after inspecting the plot
my_bad_channels = ["PO7", "O2", "PZ", "FP1", "PO8", "C2", "CB2"]

# Interpolate bad channels
all_epochs.info['bads'] = my_bad_channels
all_epochs.interpolate_bads(reset_bads=False)

# Re-reference the data to average reference
all_epochs.set_eeg_reference('average', projection=True)

# ICA for artifact removal
ica = ICA(n_components=len(all_epochs.info['ch_names']) - 1, method='fastica', random_state=42)

In [ ]:
ica.fit(all_epochs)

# Plot ICA components to identify artifacts (e.g., eye blinks)
ica.plot_components(inst=all_epochs)

# List bad components manually after inspecting the plot
my_bad_components = [0, 1, 4, 11, 15, 38, 46, 43, 56]  # Example bad components

# Remove the bad components
ica.exclude = my_bad_components
ica.apply(all_epochs)

# Save the final preprocessed data
save_name = f'data_final_{participant_id}.fif'
all_epochs.save(os.path.join(my_save_directory, save_name), overwrite=True)

# Visualization of the final data to check artifacts removal
all_epochs.plot(n_channels=64, scalings='auto', block=True)

print(f"Final preprocessed data saved as {save_name} in {my_save_directory}")